In [3]:
import sys
if '../' not in sys.path:
    sys.path.append('../')

import mplusdb
import plotly.graph_objects as go
import pandas as pd
from scipy import stats 
import importlib

In [2]:
importlib.reload(blizzcolors)
blizz_colors = blizzcolors.BlizzColors()
blizz_colors.get_color('warlock')

'#8787ED'

In [4]:
mdb = mplusdb.MplusDatabase('../.db_config')

In [5]:
spec_table = mdb.get_utility_table('spec')

In [12]:
pd.DataFrame([['death knight', 6, 'blood', 250 ,'tank', 'death_knight_blood'],
['death knight', 6 ,'frost', 251, 'damage', 'death_knight_frost']])

,0,1,2,3,4,5
0,death knight,6,blood,250,tank,death_knight_blood
1,death knight,6,frost,251,damage,death_knight_frost


In [23]:
for row in spec_table.values:
    print("['{class_name}', {class_id}, '{spec_name}', {spec_id}, '{role}', '{token}'],".format(
        class_name = row[0], class_id = row[1], spec_name = row[2], spec_id = row[3], role = row[4],
        token = row[5]))

['death knight', 6, 'blood', 250, 'tank', 'death_knight_blood'],
['death knight', 6, 'frost', 251, 'damage', 'death_knight_frost'],
['death knight', 6, 'unholy', 252, 'damage', 'death_knight_unholy'],
['demon hunter', 12, 'havoc', 577, 'damage', 'demon_hunter_havoc'],
['demon hunter', 12, 'vengeance', 581, 'tank', 'demon_hunter_vengeance'],
['druid', 11, 'balance', 102, 'damage', 'druid_balance'],
['druid', 11, 'feral', 103, 'damage', 'druid_feral'],
['druid', 11, 'guardian', 104, 'tank', 'druid_guardian'],
['druid', 11, 'restoration', 105, 'healer', 'druid_restoration'],
['hunter', 3, 'beast mastery', 253, 'damage', 'hunter_beast_mastery'],
['hunter', 3, 'marksmanship', 254, 'damage', 'hunter_marksmanship'],
['hunter', 3, 'survival', 255, 'damage', 'hunter_survival'],
['mage', 8, 'arcane', 62, 'damage', 'mage_arcane'],
['mage', 8, 'fire', 63, 'damage', 'mage_fire'],
['mage', 8, 'frost', 64, 'damage', 'mage_frost'],
['monk', 10, 'brewmaster', 268, 'tank', 'monk_brewmaster'],
['monk', 1

In [5]:
spec_role = dict(zip(spec_table.spec_id, spec_table.spec_role))

In [6]:
#cursor = conn.cursor()
#cursor.execute('use keyruns;')
#cursor.execute('select level, count(level) from temp_run where druid_restoration=1 group by level order by level;')
#ret = cursor.fetchall()
#cursor.close()
#conn.close()

In [164]:
conn = mdb.connect()

In [163]:
query = """SELECT spec, level, count(level) FROM 
    (SELECT run_id, spec, level FROM temp_run INNER JOIN
    roster ON roster.run_id = temp_run.id) as J 
    GROUP BY spec, level;
    """
print(query)


    select spec, level, count(level) from
    (select run_id, spec, level from temp_run inner join roster on roster.run_id = temp_run.id) as J 
    group by spec, level;
    


In [165]:
cursor = conn.cursor()
cursor.execute('use keyruns;')
cursor.execute(query)
ret = cursor.fetchall()
cursor.close()
conn.close()

In [170]:
df = pd.DataFrame(ret, columns = ['spec', 'key_level', 'num_keys'])
df.to_pickle('keynums_groupby_level_spec.pkl')

---

In [7]:
df = pd.read_pickle('keynums_groupby_level_spec.pkl')
table = pd.pivot_table(df,
    values='num_keys', index=['spec'],
    columns=['key_level'],
    fill_value=0)

In [8]:
sorted_index = table.sum(axis=1).sort_values(ascending=False).index

In [9]:
def hextofloats(h):
    '''Takes a hex rgb string (e.g. #ffffff) and returns an RGB tuple (float, float, float).'''
    return 'rgba(%s,%s,%s, 0.9)' % tuple(int(h[i:i + 2], 16) for i in (1, 3, 5)) # skip '#'
hextofloats('#C41F3B')

'rgba(196,31,59, 0.9)'

In [10]:
table

key_level,2,3,4,5,6,7,8,9,10,11,...,22,23,24,25,26,27,28,29,30,31
spec,,,,,,,,,,,,,,,,,,,,,
62,60866,18014,29543,44106,42838,39428,39559,39031,38790,33202,...,103,26,7,0,0,0,0,0,0,0
63,189077,52194,92874,155202,161671,147111,164569,175010,190438,201742,...,65408,44696,27731,15920,7578,2876,939,229,26,4
64,126240,37110,64627,109142,114274,107032,119819,130634,143541,141865,...,2913,1814,1145,791,466,226,71,17,3,0
65,195234,53586,95306,166004,169984,156348,180617,192424,218078,231482,...,46560,28298,15384,7452,3279,1226,371,101,10,2
66,213854,62689,108337,174370,179998,172065,192232,208725,226644,225399,...,12548,6726,3206,1360,483,132,18,1,0,0
70,195989,56146,97133,158305,159457,150835,165402,176784,200958,196296,...,8120,3720,1651,704,200,36,3,0,0,0
71,91688,25195,45483,77676,74852,72164,82703,86038,90709,93890,...,10513,5973,3243,1565,541,104,9,1,0,0
72,230357,64698,113109,195269,207902,190600,221337,241876,266611,280647,...,19214,8418,3099,951,199,29,11,2,0,0
73,206967,58323,103858,179901,189169,168443,191492,203910,231125,244085,...,61906,41647,25771,14702,6984,2681,892,220,25,3


In [207]:
def construct_trace(x, y, offset, spec_color):
    trace = {
        "fill": "tozerox", 
        "fillcolor": spec_color,
        "line": {
            "color": "black", 
            "shape": "spline", 
            "width": 1.5,
        },
        "y": y,
        "x": x
    }
    
    trace_base = {
    "line": {
        "width": 0.5,
        "color": "black"
    },
    "y": [offset for i in y],
    "x": x
    }

    return trace, trace_base

In [1]:




def get_spec_color(spec_id):
    class_name = spec_table.loc[spec_table.spec_id == spec_id, 'class_name'].values[0]
    hexc = blizz_colors.get_color(class_name.replace(' ', '_'))
    return hextofloats(hexc)
    
traces = []
annotations = []
for i, spec_id in enumerate(sorted_index):

    
    key_level = list(table)
    num_runs = table.loc[table.index==spec_id, :].values[0]
    vertical_offset = 300000*(len(sorted_index)-i)
    num_runs = [y + vertical_offset for y in num_runs]
    
    spec_color = get_spec_color(spec_id)
    trace = construct_trace(key_level, num_runs, vertical_offset, spec_color)
    traces.extend(trace)
    
    
    max_key_level = max(key_level)
    for n in num_runs[::-1]:
        if n-vertical_offset == 0:
            max_key_level -= 1
        else:
            break
    
    anno = construct_annotation(x=max_key_level, y=vertical_offset, text=max_key_level)
    annotations.append(anno)
    
fig = go.Figure()
for trace in traces:
    fig.add_trace(trace)

fig.update_layout(
    title="Distribution of runs per key level",
    xaxis_title="Key Level",
    yaxis_title="Number of Runs",
    legend_title="Legend Title",
    font=dict(
        family="Monaco, regular",
        size=16,
        color="Black"
    ),
    xaxis={'range':[2,32]},
    yaxis={'range':[0,12_500_000]},

    width=900,
    height=1500,
    showlegend=False
    #yaxis_type="log"
)

#fig.update_layout(annotations=annotations)

#button
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            buttons=[
                dict(label="Hide Highest Key",
                     method="relayout",
                     args=["annotations", []]),
                dict(label="Show Highest Key",
                     method="relayout",
                     args=["annotations", annotations]),
                dict(label="TEST_DATA_RESTYLE",
                     method="restyle",
                     args=["data", [[]]])
            ],
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.11,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

fig.show()


NameError: name 'sorted_index' is not defined

In [209]:
sorted_summary = summary_table.sort_values(by=['best_key', 'total_run'], ascending=False)
for row in list(sorted_summary.values):
    spec_id = row[0]
    total_runs = row[1]
    best_key = row[2]


In [215]:
def construct_annotation2(x, y, text):
        
    annotation = dict(
            x=x,
            y=y+50000,
            text='<b>%s</b>' % text.upper(),
            showarrow=False,
            font=dict(
                color="black",
                size=15,
                family="Monaco, regular"
            ),
            xanchor = 'right',
            yanchor = 'center',
            opacity=0.6
    )
        
    return annotation


def construct_annotation(x, y, text):
    annotation = dict(
            x=x,
            y=y,
            text=text,
            showarrow=True,
            ax=0,
            ay=-20,
            font=dict(
                color="black",
                size=15,
                family="Monaco, regular"
            ),
            #arrowcolor="black",
            arrowcolor = 'rgba(0,0,0, 0.75)',
            arrowsize=3,
            arrowwidth=1,
            arrowhead=1,
    )
    
    return annotation

In [221]:
traces = []
annotations = []

trace_tanks = []
trace_healers = []
trace_damage = []

sorted_summary = summary_table.sort_values(by=['best_key', 'total_run'], ascending=False)
#sorted_summary = summary_table.sort_values(by=['total_run', 'best_key'], ascending=False)


gray_color = 'rgba(220, 220, 220, 0.3)'
trace_colors = []
tank_colored = []
healer_colored = []
dps_colored = []


base_lines = []
xs = []


for sorted_index, row in enumerate(list(sorted_summary.values)):
    spec_id = row[0]
    total_runs = row[1]
    max_key_level = row[2]

    key_level = [i - 2 for i in list(table)]
    num_runs = table.loc[table.index==spec_id, :].values[0]
    vertical_offset = 300000*(len(sorted_summary)-sorted_index)
    num_runs = [y + vertical_offset for y in num_runs]
    
    spec_color = get_spec_color(spec_id)
    trace, trc_baseline = construct_trace(key_level, num_runs, vertical_offset, spec_color)
    base_lines.append(trc_baseline)
    
    xs.append(trace['y'])
    xs.append(trc_baseline['y'])
    
    role = spec_role[spec_id]
    if role == 'tank':
        tank_colored.append(spec_color)
        tank_colored.append(0)
    else:
        tank_colored.append(gray_color)
        tank_colored.append(0)
    
    if role == 'healer':
        healer_colored.append(spec_color)
        healer_colored.append(0)
    else:
        healer_colored.append(gray_color)
        healer_colored.append(0)
        
    if role == 'damage':
        dps_colored.append(spec_color)
        dps_colored.append(0)

    else:
        dps_colored.append(gray_color)        
        dps_colored.append(0)

        
    trace_colors.append(spec_color)
    trace_colors.append(0)
    traces.append(trace)
    traces.append(trc_baseline)
    
    anno = construct_annotation(x=max_key_level-2, y=vertical_offset, text='+%d ' % max_key_level)
    
    
    spec_name = spec_table.loc[spec_table.spec_id == spec_id, 'spec_name'].values[0]
    
    anno1 = construct_annotation2(x=0, y=vertical_offset, text= spec_name)

    annotations.append(anno)
    annotations.append(anno1)


    
#traces.extend(trace_tanks)
fig = go.Figure()
c = 0

#fig.add_trace(go.Scatter(x=[2,2], y=[0, 12_000_000], line_color = 'red'))
for trace in traces:
    #print(trace)
    fig.add_trace(trace)
    c += 1
    #if c == 3:
    #    break

#for trc in base_lines:
    #fig.add_trace(trc)
    
fig.update_layout(
    title="Distribution of runs per key level",
    xaxis_title="Key Level",
    #yaxis_title="Number of Runs",
    legend_title="Legend Title",
    font=dict(
        family="Monaco, regular",
        size=16,
        color="Black"
    ),
    xaxis={'range':[-7,32], 'tickvals': [3, 8, 13, 18, 23, 28],
           'ticktext': ['5', '10', '15', '20', '25', '30']},
    yaxis={'range':[0,12_500_000], 'showticklabels': False},
    
            
        

    width=900,
    height=1500,
    showlegend=False
    #yaxis_type="log"
)

#fig.update_layout(annotations=annotations)

#button
def blah(trace_colors):
    return trace_colors

repl = [i-500000 for i in xs[0]]
xs[0] = repl
        
        
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{"fillcolor": blah(trace_colors)}],
                    label="All",
                    method="restyle"
                ),
                dict(
                    args=[{'fillcolor': tank_colored}],
                    label="Tanks",
                    method="restyle"
                ),
                dict(
                    args=[{'fillcolor': healer_colored}],
                    label="Healers",
                    method="restyle"
                ),
                dict(
                    args=[{'fillcolor': dps_colored}],
                    label="DPS",
                    method="restyle"
                ),

            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.11,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
        dict(
            type="buttons",
            direction="right",
            buttons=[
                dict(label="Hide Highest Key",
                     method="relayout",
                     args=["annotations", []]),
                dict(label="Show Highest Key",
                     method="relayout",
                     args=["annotations", annotations])
            ],
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.11,
            xanchor="left",
            y=1,
            yanchor="top"
        )
    ]
)

#                    args=[{"contours.showlines": False, "type": "contour"}],

print(trace_colors)
fig.show()

['rgba(163,48,201, 0.9)', 0, 'rgba(169,210,113, 0.9)', 0, 'rgba(255,125,10, 0.9)', 0, 'rgba(199,156,110, 0.9)', 0, 'rgba(245,140,186, 0.9)', 0, 'rgba(64,199,235, 0.9)', 0, 'rgba(255,245,105, 0.9)', 0, 'rgba(255,125,10, 0.9)', 0, 'rgba(0,255,150, 0.9)', 0, 'rgba(64,199,235, 0.9)', 0, 'rgba(199,156,110, 0.9)', 0, 'rgba(245,140,186, 0.9)', 0, 'rgba(0,255,150, 0.9)', 0, 'rgba(135,135,237, 0.9)', 0, 'rgba(255,245,105, 0.9)', 0, 'rgba(0,112,222, 0.9)', 0, 'rgba(196,31,59, 0.9)', 0, 'rgba(255,255,255, 0.9)', 0, 'rgba(0,255,150, 0.9)', 0, 'rgba(199,156,110, 0.9)', 0, 'rgba(0,112,222, 0.9)', 0, 'rgba(255,255,255, 0.9)', 0, 'rgba(196,31,59, 0.9)', 0, 'rgba(135,135,237, 0.9)', 0, 'rgba(163,48,201, 0.9)', 0, 'rgba(196,31,59, 0.9)', 0, 'rgba(245,140,186, 0.9)', 0, 'rgba(255,125,10, 0.9)', 0, 'rgba(255,125,10, 0.9)', 0, 'rgba(0,112,222, 0.9)', 0, 'rgba(135,135,237, 0.9)', 0, 'rgba(255,255,255, 0.9)', 0, 'rgba(169,210,113, 0.9)', 0, 'rgba(169,210,113, 0.9)', 0, 'rgba(255,245,105, 0.9)', 0, 'rgba(64,1

In [115]:
print(fig)
fig.write_html("test.html")

Figure({
    'data': [{'fill': 'tozerox',
              'fillcolor': 'rgba(163,48,201, 0.9)',
              'line': {'color': 'black', 'shape': 'spline', 'width': 1.5},
              'type': 'scatter',
              'x': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
                    19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
              'y': [11206531, 10913352, 11003750, 11157985, 11183932, 11157314,
                    11214758, 11255849, 11326137, 11355852, 11343084, 11241890,
                    11495283, 12126178, 11577745, 11286436, 11161540, 11082444,
                    11026932, 10942434, 10886486, 10850477, 10826452, 10812614,
                    10804998, 10801617, 10800402, 10800081, 10800013, 10800002]},
             {'line': {'color': 'black', 'width': 0.5},
              'type': 'scatter',
              'x': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
                    19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
 

In [13]:
import plotly.graph_objects as go

# Generate dataset
import numpy as np
np.random.seed(1)

x0 = np.random.normal(2, 0.4, 400)
y0 = np.random.normal(2, 0.4, 400)
x1 = np.random.normal(3, 0.6, 600)
y1 = np.random.normal(6, 0.4, 400)
x2 = np.random.normal(4, 0.2, 200)
y2 = np.random.normal(4, 0.4, 200)

# Create figure
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(
        x=x0,
        y=y0,
        mode="markers",
        marker=dict(color="DarkOrange")
    )
)

fig.add_trace(
    go.Scatter(
        x=x1,
        y=y1,
        mode="markers",
        marker=dict(color="Crimson")
    )
)

fig.add_trace(
    go.Scatter(
        x=x2,
        y=y2,
        mode="markers",
        marker=dict(color="RebeccaPurple")
    )
)

# Add buttons that add shapes
cluster0 = [dict(type="circle",
                            xref="x", yref="y",
                            x0=min(x0), y0=min(y0),
                            x1=max(x0), y1=max(y0),
                            line=dict(color="DarkOrange"))]
cluster1 = [dict(type="circle",
                            xref="x", yref="y",
                            x0=min(x1), y0=min(y1),
                            x1=max(x1), y1=max(y1),
                            line=dict(color="Crimson"))]
cluster2 = [dict(type="circle",
                            xref="x", yref="y",
                            x0=min(x2), y0=min(y2),
                            x1=max(x2), y1=max(y2),
                            line=dict(color="RebeccaPurple"))]

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(label="None",
                     method="relayout",
                     args=["shapes", []]),
                dict(label="Cluster 0",
                     method="relayout",
                     args=["shapes", cluster0]),
                dict(label="Cluster 1",
                     method="relayout",
                     args=["shapes", cluster1]),
                dict(label="Cluster 2",
                     method="relayout",
                     args=["shapes", cluster2]),
                dict(label="All",
                     method="relayout",
                     args=["shapes", cluster0 + cluster1 + cluster2])
            ],
        )
    ]
)

# Update remaining layout properties
fig.update_layout(
    title_text="Highlight Clusters",
    showlegend=False,
)

fig.show()

In [14]:
table.head(5)

key_level,2,3,4,5,6,7,8,9,10,11,...,22,23,24,25,26,27,28,29,30,31
spec,,,,,,,,,,,,,,,,,,,,,
62,60866,18014,29543,44106,42838,39428,39559,39031,38790,33202,...,103,26,7,0,0,0,0,0,0,0
63,189077,52194,92874,155202,161671,147111,164569,175010,190438,201742,...,65408,44696,27731,15920,7578,2876,939,229,26,4
64,126240,37110,64627,109142,114274,107032,119819,130634,143541,141865,...,2913,1814,1145,791,466,226,71,17,3,0
65,195234,53586,95306,166004,169984,156348,180617,192424,218078,231482,...,46560,28298,15384,7452,3279,1226,371,101,10,2
66,213854,62689,108337,174370,179998,172065,192232,208725,226644,225399,...,12548,6726,3206,1360,483,132,18,1,0,0


In [15]:
def find_highest_key(x):
    highest_key = None
    for i, a in enumerate(x):
        if a != 0:
            highest_key = i+2
    return highest_key

table.apply(lambda x: find_highest_key(x) , axis=1)

spec
62     24
63     31
64     30
65     31
66     29
70     28
71     29
72     29
73     31
102    28
103    28
104    31
105    31
250    28
251    29
252    29
253    31
254    25
255    25
256    29
257    25
258    29
259    29
260    31
261    25
262    27
263    29
264    29
265    26
266    29
267    29
268    30
269    29
270    29
577    31
581    28
dtype: int64

In [16]:
summary_table = pd.DataFrame(table.index)
summary_table['total_run'] = table.sum(axis=1).values
summary_table['best_key'] = table.apply(lambda x: find_highest_key(x) , axis=1).values

In [17]:
summary_table.sort_values(by=['best_key', 'total_run'])

,spec,total_run,best_key
0,62,517345,24
24,261,83758,25
18,255,234320,25
17,254,465480,25
20,257,1957893,25
28,265,280676,26
25,262,1500250,27
10,103,833578,28
9,102,2450098,28
5,70,3018444,28
